In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


C:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-07_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1536
count_after above threshold (0.75): 1169
Percentage above threshold (0.75): 76.11%

df_data.shape: (1169, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2725910.0000,1.3000,22.7600,-27.5400,-18.9513,-18.3100,-24.1000,-25.4400,-19.5900,7.4900,-15.3100,-20.4800,-20.7100,-27.4200,-3.1400,-30.2300,10.6000,8.5600,4.7173,3.6400,2.9600,157.5177,54.5700,2.8900,181.4600,-3.6700,-34.1343,-15.0801,0.0000,-18.7987,-12.8173,0.0147,-9.1883,-8.4353,0.1552,-4.9401,-5.0753,0.3796,-5.9728,-5.9605,0.3298,-2.9234,-3.2515,0.5835,-1.7229,-1.9777,0.7242,0.2576,0.3418,1.0480
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2660320.0000,0.9500,28.9700,-15.1000,-6.3537,-4.6700,-9.0100,-15.4700,-14.0900,-14.3600,-6.4100,-10.1000,-14.8800,-20.1900,-0.4500,-23.5900,-0.4500,10.6600,2.9788,2.7800,2.1800,49.9717,23.6900,2.1100,357.8600,-0.5500,-15.4525,-12.8435,0.0000,-15.8809,-12.0021,0.0000,-10.3029,-9.4672,0.1637,-4.9281,-5.3380,0.4243,-3.5441,-4.2384,0.5543,-2.4739,-3.0639,0.6422,-1.3776,-1.6973,0.7790,-0.8384,-1.0542,0.8659
NVDA,NVIDIA Corp,"Technology, Semiconductors",2382420.0000,2.0800,33.1900,-27.2900,-11.5740,-9.9100,-13.3600,-32.4200,-20.5200,13.6600,-13.2100,-18.9100,-23.0400,-34.4600,6.0000,-36.2400,29.1400,6.7200,6.8824,5.6200,4.3400,606.1242,286.6800,2.1100,97.6400,3.5300,-3.9776,-5.8738,0.4767,-8.0685,-8.4554,0.2464,-9.1033,-8.6352,0.2078,-5.0398,-5.4015,0.4105,-3.0910,-3.5991,0.6070,-1.7685,-2.1198,0.7388,-1.0989,-1.3652,0.8292,0.4580,0.6305,1.0789
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1857360.0000,1.2800,32.0400,-20.1100,-10.5862,-7.8800,-12.0400,-21.8300,-3.6800,-2.6300,-9.4200,-16.9800,-12.4500,-27.7300,5.5800,-27.7300,15.6000,9.0100,5.1409,4.8900,3.3300,108.6942,42.8300,2.5400,175.2600,2.4900,-2.8615,-4.5605,0.5937,-6.3296,-6.9703,0.3386,-7.4216,-7.3608,0.2639,-3.0463,-3.4291,0.5785,-3.8145,-4.2106,0.5307,-2.6468,-3.0492,0.6384,-0.3967,-0.5149,0.9359,-0.1704,-0.2246,0.9717
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1804770.0000,0.9900,30.9000,-21.6300,-6.0556,-4.4700,-15.0800,-22.7300,-10.7500,-1.7800,-8.1000,-15.3400,-15.1200,-28.4900,1.1500,-28.4900,1.1500,5.6100,3.7590,2.8300,2.7700,47.4666,20.5300,2.3100,149.2400,1.0200,-5.9070,-7.7406,0.3104,-10.2228,-9.6536,0.1391,-10.7747,-9.5479,0.1610,-3.9401,-4.4441,0.5313,-4.2246,-4.7152,0.5269,-3.1932,-3.6436,0.5934,-0.6683,-0.8863,0.8956,-0.1738,-0.2377,0.9705


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000
mean,55795.1839,0.9854,31.6934,-9.7945,-10.7009,-9.6066,-11.1067,-10.6564,-9.1440,0.8547,-9.2417,-11.8859,-10.4794,-20.9440,3.7012,-25.8106,20.1188,3.8673,4.2222,3.8480,2.9341,14.5787,6.2988,2.2432,92.7660,-0.9005,-45.1334,-7.5405,0.4925,-10.4741,-8.9140,0.2238,-6.7032,-6.1521,0.3653,-4.3357,-4.3223,0.4948,-2.8242,-3.0458,0.6438,-0.9287,-0.9384,0.8842,-0.7968,-0.9095,0.8823,-0.0398,0.0385,1.0060
std,182071.9746,0.5524,9.1425,15.2119,5.8863,5.6345,8.6759,15.5463,19.0709,42.6303,6.0163,9.1929,13.2462,12.0477,6.7114,14.7066,47.6077,5.0695,2.4419,2.0954,1.6055,40.0973,14.0789,1.3199,105.5778,2.5414,177.8013,46.3540,4.1070,5.3190,5.0132,0.4826,3.8612,4.4720,0.4883,2.8749,3.1203,0.3557,2.0391,2.3764,0.2804,1.7355,2.3005,0.2997,1.1285,1.4911,0.1918,0.8615,1.2749,0.1653
min,2980.0000,-3.3800,9.1100,-72.6100,-43.7269,-42.6300,-56.1800,-73.2300,-74.0100,-84.1800,-47.0300,-58.9500,-72.3800,-73.8100,-6.3600,-95.3300,-6.3600,0.0200,0.0359,0.0100,0.0100,0.7550,0.7501,0.8000,2.1100,-10.8500,-3833.2595,-15.8744,0.0000,-45.4689,-15.1951,0.0000,-17.6704,-12.1364,0.0000,-14.8588,-11.1668,0.0542,-7.3675,-7.4273,0.1731,-5.6580,-5.9760,0.2590,-3.8979,-4.3180,0.4801,-2.6324,-3.0522,0.6189
25%,8340.0000,0.6600,24.9200,-19.2300,-14.1755,-12.6300,-16.1000,-20.3700,-19.2100,-16.3500,-12.8000,-17.4100,-18.2800,-28.0600,-0.3200,-35.3100,1.7900,0.9600,2.7730,2.6200,2.0500,2.8640,1.5000,1.5400,31.5700,-2.4100,-27.7530,-14.7164,0.0000,-13.6961,-11.2679,0.0468,-9.3100,-8.5536,0.1321,-6.0179,-6.1847,0.2833,-4.3306,-4.7055,0.4455,-2.2161,-2.5839,0.6643,-1.6671,-2.0167,0.7393,-0.6103,-0.7965,0.8963
50%,16910.0000,0.9700,30.0800,-9.7300,-10.7720,-9.8100,-11.7800,-10.5800,-10.6600,-2.6900,-9.5100,-11.8300,-10.6600,-19.2700,1.9400,-23.9300,7.3600,2.3100,3.7427,3.4800,2.7200,5.3756,2.6200,1.9100,59.3500,-1.0300,-15.5321,-12.8663,0.0000,-10.4640,-9.7546,0.1217,-7.3238,-7.2577,0.2288,-4.5844,-4.8617,0.4045,-3.1780,-3.6444,0.5688,-1.2176,-1.5195,0.8040,-0.8808,-1.1255,0.8519,-0.1080,-0.1448,0.9806
75%,43920.0000,1.2300,37.3100,-0.7000,-7.2933,-6.2800,-5.1600,-0.7100,-0.5800,10.1300,-5.2200,-5.3800,-1.7100,-12.6300,5.8500,-15.0000,23.7000,4.9100,5.2838,4.8300,3.6600,12.7096,5.5100,2.5000,108.0500,0.2200,-9.5430,-10.3159,0.0810,-6.9504,-7.5598,0.2824,-4.6706,-5.1172,0.4192,-2.7415,-3.2188,0.6031,-1.5471,-1.8770,0.7642,0.2371,0.3043,1.0400,-0.0568,-0.0786,0.9896,0.4797,0.6598,1.0901
max,2725910.0000,4.2500,79.8800,61.1100,37.0765,30.8600,45.5500,68.1300,145.9500,848.0000,30.7000,50.5900,50.8000,-0.0400,91.3800,-0.0400,971.8600,44.6200,25.2459,18.8900,14.9700,716.6563,286.6800,26.9800,908.1300,16.2200,144.7788,513.9591,46.7871,11.7184,110.5006,13.0365,10.1651,72.0544,9.9401,6.6845,20.9286,3.6986,4.8525,12.5112,2.3557,4.4300,9.7330,2.2790,2.5016,5.7738,1.8774,3.7578,7.5376,1.8147


In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-07
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Appending new entry for 2025-04-07
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-07


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000,1169.0000
mean,55795.1839,0.9854,31.6934,-9.7945,-10.7009,-9.6066,-11.1067,-10.6564,-9.1440,0.8547,-9.2417,-11.8859,-10.4794,-20.9440,3.7012,-25.8106,20.1188,3.8673,4.2222,3.8480,2.9341,14.5787,6.2988,2.2432,92.7660,-0.9005,-45.1334,-7.5405,0.4925,-10.4741,-8.9140,0.2238,-6.7032,-6.1521,0.3653,-4.3357,-4.3223,0.4948,-2.8242,-3.0458,0.6438,-0.9287,-0.9384,0.8842,-0.7968,-0.9095,0.8823,-0.0398,0.0385,1.0060
std,182071.9746,0.5524,9.1425,15.2119,5.8863,5.6345,8.6759,15.5463,19.0709,42.6303,6.0163,9.1929,13.2462,12.0477,6.7114,14.7066,47.6077,5.0695,2.4419,2.0954,1.6055,40.0973,14.0789,1.3199,105.5778,2.5414,177.8013,46.3540,4.1070,5.3190,5.0132,0.4826,3.8612,4.4720,0.4883,2.8749,3.1203,0.3557,2.0391,2.3764,0.2804,1.7355,2.3005,0.2997,1.1285,1.4911,0.1918,0.8615,1.2749,0.1653
min,2980.0000,-3.3800,9.1100,-72.6100,-43.7269,-42.6300,-56.1800,-73.2300,-74.0100,-84.1800,-47.0300,-58.9500,-72.3800,-73.8100,-6.3600,-95.3300,-6.3600,0.0200,0.0359,0.0100,0.0100,0.7550,0.7501,0.8000,2.1100,-10.8500,-3833.2595,-15.8744,0.0000,-45.4689,-15.1951,0.0000,-17.6704,-12.1364,0.0000,-14.8588,-11.1668,0.0542,-7.3675,-7.4273,0.1731,-5.6580,-5.9760,0.2590,-3.8979,-4.3180,0.4801,-2.6324,-3.0522,0.6189
25%,8340.0000,0.6600,24.9200,-19.2300,-14.1755,-12.6300,-16.1000,-20.3700,-19.2100,-16.3500,-12.8000,-17.4100,-18.2800,-28.0600,-0.3200,-35.3100,1.7900,0.9600,2.7730,2.6200,2.0500,2.8640,1.5000,1.5400,31.5700,-2.4100,-27.7530,-14.7164,0.0000,-13.6961,-11.2679,0.0468,-9.3100,-8.5536,0.1321,-6.0179,-6.1847,0.2833,-4.3306,-4.7055,0.4455,-2.2161,-2.5839,0.6643,-1.6671,-2.0167,0.7393,-0.6103,-0.7965,0.8963
50%,16910.0000,0.9700,30.0800,-9.7300,-10.7720,-9.8100,-11.7800,-10.5800,-10.6600,-2.6900,-9.5100,-11.8300,-10.6600,-19.2700,1.9400,-23.9300,7.3600,2.3100,3.7427,3.4800,2.7200,5.3756,2.6200,1.9100,59.3500,-1.0300,-15.5321,-12.8663,0.0000,-10.4640,-9.7546,0.1217,-7.3238,-7.2577,0.2288,-4.5844,-4.8617,0.4045,-3.1780,-3.6444,0.5688,-1.2176,-1.5195,0.8040,-0.8808,-1.1255,0.8519,-0.1080,-0.1448,0.9806
75%,43920.0000,1.2300,37.3100,-0.7000,-7.2933,-6.2800,-5.1600,-0.7100,-0.5800,10.1300,-5.2200,-5.3800,-1.7100,-12.6300,5.8500,-15.0000,23.7000,4.9100,5.2838,4.8300,3.6600,12.7096,5.5100,2.5000,108.0500,0.2200,-9.5430,-10.3159,0.0810,-6.9504,-7.5598,0.2824,-4.6706,-5.1172,0.4192,-2.7415,-3.2188,0.6031,-1.5471,-1.8770,0.7642,0.2371,0.3043,1.0400,-0.0568,-0.0786,0.9896,0.4797,0.6598,1.0901
max,2725910.0000,4.2500,79.8800,61.1100,37.0765,30.8600,45.5500,68.1300,145.9500,848.0000,30.7000,50.5900,50.8000,-0.0400,91.3800,-0.0400,971.8600,44.6200,25.2459,18.8900,14.9700,716.6563,286.6800,26.9800,908.1300,16.2200,144.7788,513.9591,46.7871,11.7184,110.5006,13.0365,10.1651,72.0544,9.9401,6.6845,20.9286,3.6986,4.8525,12.5112,2.3557,4.4300,9.7330,2.2790,2.5016,5.7738,1.8774,3.7578,7.5376,1.8147


In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-07


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2725910.0000,1.3000,22.7600,-27.5400,-18.9513,-18.3100,-24.1000,-25.4400,-19.5900,7.4900,-15.3100,-20.4800,-20.7100,-27.4200,-3.1400,-30.2300,10.6000,8.5600,4.7173,3.6400,2.9600,157.5177,54.5700,2.8900,181.4600,-3.6700,-34.1343,-15.0801,0.0000,-18.7987,-12.8173,0.0147,-9.1883,-8.4353,0.1552,-4.9401,-5.0753,0.3796,-5.9728,-5.9605,0.3298,-2.9234,-3.2515,0.5835,-1.7229,-1.9777,0.7242,0.2576,0.3418,1.0480
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2660320.0000,0.9500,28.9700,-15.1000,-6.3537,-4.6700,-9.0100,-15.4700,-14.0900,-14.3600,-6.4100,-10.1000,-14.8800,-20.1900,-0.4500,-23.5900,-0.4500,10.6600,2.9788,2.7800,2.1800,49.9717,23.6900,2.1100,357.8600,-0.5500,-15.4525,-12.8435,0.0000,-15.8809,-12.0021,0.0000,-10.3029,-9.4672,0.1637,-4.9281,-5.3380,0.4243,-3.5441,-4.2384,0.5543,-2.4739,-3.0639,0.6422,-1.3776,-1.6973,0.7790,-0.8384,-1.0542,0.8659
NVDA,NVIDIA Corp,"Technology, Semiconductors",2382420.0000,2.0800,33.1900,-27.2900,-11.5740,-9.9100,-13.3600,-32.4200,-20.5200,13.6600,-13.2100,-18.9100,-23.0400,-34.4600,6.0000,-36.2400,29.1400,6.7200,6.8824,5.6200,4.3400,606.1242,286.6800,2.1100,97.6400,3.5300,-3.9776,-5.8738,0.4767,-8.0685,-8.4554,0.2464,-9.1033,-8.6352,0.2078,-5.0398,-5.4015,0.4105,-3.0910,-3.5991,0.6070,-1.7685,-2.1198,0.7388,-1.0989,-1.3652,0.8292,0.4580,0.6305,1.0789
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1857360.0000,1.2800,32.0400,-20.1100,-10.5862,-7.8800,-12.0400,-21.8300,-3.6800,-2.6300,-9.4200,-16.9800,-12.4500,-27.7300,5.5800,-27.7300,15.6000,9.0100,5.1409,4.8900,3.3300,108.6942,42.8300,2.5400,175.2600,2.4900,-2.8615,-4.5605,0.5937,-6.3296,-6.9703,0.3386,-7.4216,-7.3608,0.2639,-3.0463,-3.4291,0.5785,-3.8145,-4.2106,0.5307,-2.6468,-3.0492,0.6384,-0.3967,-0.5149,0.9359,-0.1704,-0.2246,0.9717
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1804770.0000,0.9900,30.9000,-21.6300,-6.0556,-4.4700,-15.0800,-22.7300,-10.7500,-1.7800,-8.1000,-15.3400,-15.1200,-28.4900,1.1500,-28.4900,1.1500,5.6100,3.7590,2.8300,2.7700,47.4666,20.5300,2.3100,149.2400,1.0200,-5.9070,-7.7406,0.3104,-10.2228,-9.6536,0.1391,-10.7747,-9.5479,0.1610,-3.9401,-4.4441,0.5313,-4.2246,-4.7152,0.5269,-3.1932,-3.6436,0.5934,-0.6683,-0.8863,0.8956,-0.1738,-0.2377,0.9705
